In [1]:
import os
import cv2
import sys
import hashlib
import numpy as np
import scipy.special as sp
import matplotlib
import matplotlib.pyplot as plt
import datetime  



In [2]:
# determine if we are using Windows or Linux filing conventions and assign root folders
if sys.platform == 'win32':
    SCAPA_ROOT = os.path.join('Z:', os.sep, 'group')
    ORKNEY_ROOT = os.path.join('U:', os.sep)
else:
    SCAPA_ROOT = '/mnt/scapa4'
    ORKNEY_ROOT  ='/mnt/orkney1'

ORKNEY_TEAM = os.path.join(ORKNEY_ROOT, 'Clusters')
ORKNEY_PROJECT = os.path.join(ORKNEY_TEAM, 'RandomXtl')
ORKNEY_MASKRCNN = os.path.join(ORKNEY_PROJECT, 'MaskRCNN')
ORKNEY_DETECTED = os.path.join(ORKNEY_MASKRCNN, 'detected')
ORKNEY_TRAINING = os.path.join(ORKNEY_MASKRCNN, 'training')
ORKNEY_DATASETS = os.path.join(ORKNEY_TRAINING, 'datasets')
ORKNEY_LOGS = os.path.join(ORKNEY_TRAINING, 'logs')

ORKNEY_IMAGES = os.path.join(ORKNEY_PROJECT, 'images')
ORKNEY_RAW_IMGS = os.path.join(ORKNEY_IMAGES, 'raw_images')
ORKNEY_PART_IMGS = os.path.join(ORKNEY_IMAGES, 'partitioned')

ORKNEY_CV = os.path.join(ORKNEY_PROJECT, 'computer_vision')
ORKNEY_VIALS = os.path.join(ORKNEY_CV, 'vials')
ORKNEY_STRINGS = os.path.join(ORKNEY_PROJECT, 'random numbers')

IMG_EXTS = ['.img', '.bmp', '.tiff', '.jpg', '.png']

In [3]:
ranges = {'W19':{'filter': [np.array([0,180,0]), np.array([180,255,255])], 'rim': 0},
          'Co4': {'filter':[np.array([0,0,120]), np.array([180,255,230])], 'rim': 42},
          'CuSO4':{'filter': [np.array([0,0,100]), np.array([180,255,250])], 'rim': 42}
         }

In [4]:
def get_vial(image_path, cmpd):
    
    image_head, image_ext = os.path.splitext(image_path)
    image_title = os.path.basename(image_head)
    
    reaction_path = os.path.dirname(os.path.dirname(image_path))
    rxn_id = os.path.basename(reaction_path)
    exp_path = os.path.dirname(reaction_path)
    exp_id = os.path.basename(exp_path)

    output_masks_dir = os.path.join(ORKNEY_VIALS, cmpd, exp_id, rxn_id, 'masks')
    os.makedirs(output_masks_dir, exist_ok=True)
    mask_path = os.path.join(output_masks_dir, image_title+'.png')
    
    output_data_dir = os.path.join(ORKNEY_VIALS, cmpd, exp_id, rxn_id, 'data')
    os.makedirs(output_data_dir, exist_ok=True)
    data_path = os.path.join(output_data_dir, image_title+'.txt')
    
    image = cv2.imread(image_path)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, ranges[cmpd]['filter'][0], ranges[cmpd]['filter'][1])
    res = cv2.bitwise_and(image, image,mask=mask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    
    contours = cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[1]
    largest_contours = sorted(contours, key=lambda x: cv2.contourArea(x))[-5:]
    

    bg = np.zeros(image.shape[:2], np.uint8)
    roi = bg.copy()
    roi[:, 200:1080] = 255
    best_overlap = 0
    best_cnt = None
    approx_vial_rim = ranges[cmpd]['rim']
    for idx, cnt in enumerate(largest_contours):  
        contour_pic = cv2.drawContours(bg.copy(), [cnt], -1, 255, -1)
        contour_overlap_pic = np.bitwise_and(contour_pic, roi)
        overlap_amount = np.sum(contour_overlap_pic)
        if overlap_amount > best_overlap:
            best_cnt = contour_overlap_pic
            best_overlap = overlap_amount
            (x,y),radius = cv2.minEnclosingCircle(cnt)
            x, y = int(x),int(y)
            radius = int(radius)
            if radius > approx_vial_rim:
                best_cnt = cv2.circle(bg.copy(), (x, y), radius-approx_vial_rim, 255, -1)

#     test = cv2.bitwise_and(image, image, mask=best_cnt)   
#     plt.imshow(test)
#     plt.show()
    cv2.imwrite(mask_path, best_cnt)
    with open(data_path, 'w') as f:
        f.write('{},{},{}'.format(x,y,radius))
# cmpd = 'Co4'
# image_path = 'U:\\Chemobot\\crystalbot_imgs\\Co4\\20180219-0\\Reaction_020\\Images\\Image_050.png'
# get_vial(image_path)

In [5]:
def run_exp(exp_path, cmpd, reaction_range=[0,-1], image_range=[0,-1]):
    start, end = reaction_range
    rxns = sorted([i for i in os.listdir(exp_path) if 'reaction' in i.lower()])
    for rxn in rxns[start:end]:
        rxn_path = os.path.join(exp_path, rxn)
        run_rxn(rxn_path, cmpd, image_range)
    
def run_rxn(rxn_path, cmpd, image_range=[0,-1]):
    start, end = image_range
    image_dir = os.path.join(rxn_path, 'Images')
    image_names = sorted([i for i in os.listdir(image_dir)])
    for name in image_names[start: end]:      
        image_path= os.path.join(image_dir, name)
        sys.stdout.write('\r{}'.format(image_path))
        data = get_vial(image_path, cmpd)


In [7]:
cmpd = 'W19'
exp_path = 'U:\\Chemobot\\crystalbot_imgs\\W19\\20180214-0'

run_exp(exp_path, cmpd=cmpd, reaction_range=[0,-1], image_range=[20,21])

U:\Chemobot\crystalbot_imgs\W19\20180214-0\Reaction_099\Images\Image_021.png